In [1]:
import numpy as np
import sys
import os
from os.path import join as pjoin


# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def mean_variance(data_dir, save_dir, joints_num):
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]

    np.save(pjoin(save_dir, 'Mean.npy'), Mean)
    np.save(pjoin(save_dir, 'Std.npy'), Std)

    return Mean, Std

In [2]:
# The given data is used to double check if you are on the right track.
reference1 = np.load('./HumanML3D/Mean.npy')
reference2 = np.load('./HumanML3D/Std.npy')

In [5]:
if __name__ == '__main__':
    data_dir = './HumanML3D/new_joint_vecs/'
    save_dir = './HumanML3D/'
    mean, std = mean_variance(data_dir, save_dir, 22)
    print(mean)
    # print(Std)

(240370, 263)
[-2.56561216e-05  7.80974005e-06  4.04973282e-03  8.17316771e-01
  6.42598569e-02  7.36244321e-01  8.73727631e-03 -6.10007718e-02
  7.41017580e-01  8.15284625e-03  2.34096326e-04  9.32489157e-01
  1.67603903e-02  1.36247039e-01  4.66520697e-01 -4.97096814e-02
 -1.35647804e-01  4.67245132e-01 -5.11373319e-02  5.42607740e-04
  1.05664647e+00 -5.09790005e-03  1.24266677e-01  1.33536890e-01
 -5.80741093e-03 -1.25072122e-01  1.27019987e-01 -6.39354903e-03
  5.98222017e-04  1.10264838e+00 -1.62985548e-02  1.73103809e-01
  4.96625863e-02 -3.88099700e-02 -1.72030762e-01  4.65114787e-02
 -3.82673182e-02  9.97056603e-04  1.29274964e+00 -2.61997879e-02
  7.13793188e-02  1.20739889e+00 -1.95092000e-02 -6.65250197e-02
  1.20262408e+00 -1.94651335e-02  1.11518952e-03  1.36117566e+00
 -5.08400351e-02  1.73045009e-01  1.22591400e+00 -2.63656825e-02
 -1.74548849e-01  1.22231209e+00 -2.68988330e-02  2.45175242e-01
  1.05841196e+00 -3.94440405e-02 -2.45464623e-01  1.05993283e+00
 -3.9879333

### Check if your data is correct. If it's aligned with the given reference, then it is right

In [6]:
abs(mean-reference1).sum()

19.48705

In [7]:
abs(std-reference2).sum()

1.7192488